In [1]:
%load_ext autoreload

In [2]:
import lightning.pytorch as pl
import torch
import pandas as pd
import os
import scanpy as sc
import seaborn as sns
import dask.dataframe as dd
import matplotlib.pyplot as plt

In [3]:
%autoreload 2
from self_supervision.estimator.cellnet import EstimatorAutoEncoder
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.tester.reconstruction.test import eval_rec, test_pretrained_model, test_models_on_ood, test_random_model

In [4]:
DATA_PATH = '/lustre/groups/ml01/workspace/till.richter/merlin_cxg_2023_05_15_sf-log1p'  # this is a shared directory in which the data is stored
MODEL_PATH = '/lustre/groups/ml01/workspace/till.richter/trained_models/'
RESULT_PATH = '/lustre/groups/ml01/workspace/till.richter/ssl_results'
HVG = False

### Initialize the lightning model

In [5]:
# init estim class
estim = EstimatorAutoEncoder(DATA_PATH, hvg=HVG)

In [6]:
# init datamodule
estim.init_datamodule(batch_size=1024) 

In [7]:
# init model
hidden_units = [512, 512, 256, 256, 64]
estim.init_model(
    model_type='mlp_ae',
    model_kwargs={
        'learning_rate': 1e-3,
        'weight_decay': 0.1,
        'lr_scheduler': torch.optim.lr_scheduler.StepLR,
        'lr_scheduler_kwargs': {
            'step_size': 2,
            'gamma': 0.9,
            'verbose': True
        },
        'units_encoder': hidden_units,
        'units_decoder': hidden_units[::-1][1:],
    },
)

In [8]:
estim.trainer = pl.Trainer(logger=[], accelerator='cpu', devices=1)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Load all models of interest

In [9]:
"""
model_dirs = [
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Only Pretrained Models ###
    # Individual Gene Masking
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50p/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Masking
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gene_program_C8_25p/default/version_2/checkpoints/best_checkpoint_val.ckpt',

    # Gene Program to Gene Program
    MODEL_PATH + '/pretext_models/masking/CN_MLP_single_gene_program/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Boostrap Your Own Latent (BYOL)
    # MODEL_PATH + '/pretext_models/contrastive/MLP_BYOL_Gaussian_0_001_v4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins
    # MODEL_PATH + '/pretext_models/contrastive/MLP_bt_Gaussian_0_001/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Individual Gene Masking
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Masking
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gene_program_C8_25prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Gene Program    
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_single_gene_programrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Boostrap Your Own Latent (BYOL)    
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_MLP_BYOL_Gaussian_0_001run0/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins    
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_contrastive_MLP_bt_Gaussian_0_01run0/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]
"""

"\nmodel_dirs = [\n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    \n    ### Only Pretrained Models ###\n    # Individual Gene Masking\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_50p/default/version_1/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pr

In [35]:

# Negative Binomial Fitting
model_dirs = [
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run1NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run2NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run3NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run4NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Only Pretrained Models ###
    # Random Mask
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_50prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_50prun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_50prun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_50prun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_50prun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Mask
    # TO DO, GOT DELETED
    
    # GP to GP
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_single_gene_programrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # GP to TF
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Random Mask
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun0NegBin_/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Mask
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gene_program_C8_25prun0NegBin_/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # GP to GP
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_single_gene_programrun0NegBin_/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # GP to TF
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # BYOL
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_MLP_BYOL_Gaussian_0_001_v4run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_contrastive_MLP_bt_Gaussian_0_01run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

### Evaluate all models

In [ ]:
metrics = eval_rec(estim, model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_50prun0
Skipping model:  CN_NegBin_50prun1
Skipping model:  CN_NegBin_50prun2
Skipping model:  CN_NegBin_50prun3
Skipping model:  CN_NegBin_50prun4
Skipping model:  CN_NegBin_single_gene_programrun0
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_MLP_50prun0NegBin_
Skipping model:  CN_SSL_CN_CN_MLP_gene_program_C8_25prun0NegBin_
Skipping model:  CN_SSL_CN_CN_MLP_single_gene_programrun0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Sk

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │     0.17851471900939941     │
│ test_explained_var_weighted │     0.3786527216434479      │
│          test_loss          │       4687.3623046875       │
│          test_mse           │     0.10032518208026886     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_MLP_BYOL_Gaussian_0_001_v4run0NegBin_  :
 [{'test_explained_var_uniform': 0.17851471900939941, 'test_explained_var_weighted': 0.3786527216434479, 'test_mse': 0.10032518208026886, 'test_loss': 4687.3623046875}]
Evaluating model:  CN_SSL_CN_contrastive_MLP_bt_Gaussian_0_01run0NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

# Fig 3: OOD Reconstruction

**Dissection: Tail of Hippocampus (HiT) - Caudal Hippocampus - CA4-DGC**

- 56,367 cells
- 10x 3' v3
- hippocampal formation
- astrocyte (3761), central nervous system macrophage (1782), endothelial cell (174), ependymal cell (111), ~~fibroblast (86)~~, leukocyte (36), neuron (36588), oligodendrocyte (11875), oligodendrocyte precursor cell (1896), pericyte (39), vascular associated smooth muscle cell (19)

In [20]:
ood_set = 'tail_of_hippocampus'

In [13]:
"""
model_dirs = [
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

"\nmodel_dirs = [\n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    \n    ### Only Pretrained Models ###\n    \n    # Gene Program to Transcription Factor    \n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpo

In [21]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.021926123648881912     │
│ test_explained_var_weighted │     0.0740605816245079      │
│          test_loss          │       6098.4521484375       │
│          test_mse           │     0.10468926280736923     │
└─────────────────────────────┴─────────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0039011482149362564    │
│ test_explained_var_weighted │   -8.644549234304577e-05    │
│          test_loss          │     0.34012100100517273     │
│          test_mse           │     0.34012100100517273     │
└─────────────────────────────┴─────────────────────────────┘

**All non-neuronal cells**

- 888,263 cells
- 10x 3' v3
- Bergmann glial cell (8041), astrocyte (155025), central nervous system macrophage (91383), ~~choroid plexus epithelial cell (7689)~~, endothelial cell (5165), ependymal cell (5882), ~~fibroblast (9156)~~, oligodendrocyte (494966), oligodendrocyte precursor cell (105734), pericyte (3693), vascular associated smooth muscle cell (1074)

In [22]:
ood_set = 'non_neuronal'

In [23]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.012672696262598038     │
│ test_explained_var_weighted │    0.040245991200208664     │
│          test_loss          │       5714.0126953125       │
│          test_mse           │     0.14170324802398682     │
└─────────────────────────────┴─────────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0014128991169854999    │
│ test_explained_var_weighted │    -8.56504266266711e-05    │
│          test_loss          │     0.37507933378219604     │
│          test_mse           │     0.37507933378219604     │
└─────────────────────────────┴─────────────────────────────┘

In [24]:
ood_set = 'cort_dev'

In [25]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.018815871328115463     │
│ test_explained_var_weighted │     0.0752025693655014      │
│          test_loss          │      5861.79541015625       │
│          test_mse           │     0.2032804787158966      │
└─────────────────────────────┴─────────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0005281642079353333    │
│ test_explained_var_weighted │   -6.892807869007811e-05    │
│          test_loss          │     0.40849241614341736     │
│          test_mse           │     0.40849241614341736     │
└─────────────────────────────┴─────────────────────────────┘

In [26]:
ood_set = 'circ_imm'

In [27]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.029800795018672943     │
│ test_explained_var_weighted │     0.0887766182422638      │
│          test_loss          │      4718.81689453125       │
│          test_mse           │     0.16482721269130707     │
└─────────────────────────────┴─────────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0005341343930922449    │
│ test_explained_var_weighted │   -7.018810720182955e-05    │
│          test_loss          │     0.4097417891025543      │
│          test_mse           │     0.4097417891025543      │
└─────────────────────────────┴─────────────────────────────┘

In [31]:
ood_set = 'great_apes'

In [32]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0011111892526969314    │
│ test_explained_var_weighted │   -7.800337334629148e-05    │
│          test_loss          │     0.2914314866065979      │
│          test_mse           │     0.2914314866065979      │
└─────────────────────────────┴─────────────────────────────┘

# Fig 4: Individual Atlas

### HLCA

In [9]:
model_dirs = [
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]

In [10]:
supervised_subset = 148  # HLCA

In [11]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0005182110471650958    │
│ test_explained_var_weighted │   -6.305475108092651e-05    │
│          test_loss          │     0.35207226872444153     │
│          test_mse           │     0.35207226872444153     │
└─────────────────────────────┴─────────────────────────────┘

In [12]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_No_SSL_CN_new_run0NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run1NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run2NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run3NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run4NegBin__HLCA
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__HLCA
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__HLCA
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.049650926142930984     │
│ test_explained_var_weighted │     0.2730991840362549      │
│          test_loss          │       4347.6298828125       │
│          test_mse           │     0.09400048106908798     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA  :
 [{'test_explained_var_uniform': 0.049650926142930984, 'test_explained_var_weighted': 0.2730991840362549, 'test_mse': 0.09400048106908798, 'test_loss': 4347.6298828125}]
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__HLCA


...

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__HLCA  :
 [{'test_explained_var_uniform': 0.0478813536465168, 'test_explained_var_weighted': 0.2738361358642578, 'test_mse': 0.09390199184417725, 'test_loss': 4345.16650390625}]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │     -0.0253374632447958     │
│ test_explained_var_weighted │   -0.00030291362782008946   │
│          test_loss          │      12285.4345703125       │
│          test_mse           │     0.5380129814147949      │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  Random  :
 [{'test_explained_var_uniform': -0.0253374632447958, 'test_explained_var_weighted': -0.00030291362782008946, 'test_mse': 0.5380129814147949, 'test_loss': 12285.4345703125}]


### PBMC

In [13]:
"""
model_dirs = [
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run2_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run3_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run4_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run1_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run2_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run3_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run4_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

"\nmodel_dirs = [\n    ### Only Pretrained Models ###\n    \n    # Gene Program to Transcription Factor    \n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    \n    \n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_PBMC/default/version_0/checkpoints/best_check

In [14]:
model_dirs = [
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt', 
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__PBMC/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

In [15]:
supervised_subset = 41  # PBMC

In [16]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -2.128240703314077e-05    │
│ test_explained_var_weighted │   -3.438305066083558e-05    │
│          test_loss          │     0.3783740699291229      │
│          test_mse           │     0.3783740699291229      │
└─────────────────────────────┴─────────────────────────────┘

In [17]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_No_SSL_CN_new_run0NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run1NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run2NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run3NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run4NegBin__PBMC
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__PBMC
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__PBMC
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__PBMC


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.025053083896636963     │
│ test_explained_var_weighted │     0.1385020762681961      │
│          test_loss          │      3710.599853515625      │
│          test_mse           │     0.11272957921028137     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__PBMC  :
 [{'test_explained_var_uniform': 0.025053083896636963, 'test_explained_var_weighted': 0.1385020762681961, 'test_mse': 0.11272957921028137, 'test_loss': 3710.599853515625}]
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__PBMC


...

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__PBMC  :
 [{'test_explained_var_uniform': 0.024785147979855537, 'test_explained_var_weighted': 0.13787758350372314, 'test_mse': 0.11281869560480118, 'test_loss': 3712.4931640625}]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    -0.02390097640454769     │
│ test_explained_var_weighted │   -0.0003632617008406669    │
│          test_loss          │      12240.8251953125       │
│          test_mse           │     0.5376090407371521      │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  Random  :
 [{'test_explained_var_uniform': -0.02390097640454769, 'test_explained_var_weighted': -0.0003632617008406669, 'test_mse': 0.5376090407371521, 'test_loss': 12240.8251953125}]


### Tabula Sapiens

In [18]:
"""
model_dirs = [
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run2_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run3_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run4_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run1_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run2_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run3_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run4_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

"\nmodel_dirs = [\n    ### Only Pretrained Models ###\n    \n    # Gene Program to Transcription Factor    \n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    \n    \n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_Tabula_Sapiens/default/version_0/ch

In [9]:
model_dirs = [
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run5NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run6NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run7NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run8NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run9NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

In [10]:
supervised_subset = 87  # Tabula Sapiens

In [21]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0005711353733204305    │
│ test_explained_var_weighted │   -6.664633110631257e-05    │
│          test_loss          │     0.3395370543003082      │
│          test_mse           │     0.3395370543003082      │
└─────────────────────────────┴─────────────────────────────┘

In [11]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_No_SSL_CN_new_run0NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run1NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run2NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run3NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run4NegBin__Tabula_Sapiens
Evaluating model:  CN_No_SSL_CN_new_run5NegBin__Tabula_Sapiens


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │     0.03699777275323868     │
│ test_explained_var_weighted │     0.24977603554725647     │
│          test_loss          │       4923.5048828125       │
│          test_mse           │     0.08711201697587967     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_No_SSL_CN_new_run5NegBin__Tabula_Sapiens  :
 [{'test_explained_var_uniform': 0.03699777275323868, 'test_explained_var_weighted': 0.24977603554725647, 'test_mse': 0.08711201697587967, 'test_loss': 4923.5048828125}]
Skipping model:  CN_No_SSL_CN_new_run6NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run7NegBin__Tabula_Sapiens
Evaluating model:  CN_No_SSL_CN_new_run8NegBin__Tabula_Sapiens


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

